In [1]:
import pandas as pd
import numpy as np
import osmnx as ox
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle
import data_preprocessing as dp
import mdp

In [2]:
url_FT_train = 'INTERACTION/DR_USA_Roundabout_FT_train.csv'
df_train = pd.read_csv(url_FT_train)

# add a new label unique_id using case_id and track_id
df_train['unique_id'] = df_train['case_id'].astype(int) * 100+ df_train['track_id']

# change length, width and psi_rad from NaN to 0 for pedestrain/bicycle
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'length'] = 0
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'width'] = 0
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'psi_rad'] = 0

### important!!!
# agent_type pedestrain/bicycle = -1
# agent_type car = 1
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'agent_type'] = -1
df_train.loc[df_train['agent_type'] == 'car', 'agent_type'] = 1
df_train

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width,unique_id
0,1.0,1,1,100,-1,1044.509,1012.175,0.103,-1.005,0.000,0.00,0.00,101
1,1.0,2,1,100,-1,1043.840,1013.348,-0.101,-1.362,0.000,0.00,0.00,102
2,1.0,3,1,100,-1,1007.638,1022.038,0.687,0.787,0.000,0.00,0.00,103
3,1.0,4,1,100,1,1036.888,976.415,-1.063,1.980,2.064,5.20,1.86,104
4,1.0,5,1,100,1,1038.206,993.477,-2.343,5.161,1.997,4.68,1.90,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243282,9811.0,8,40,4000,1,1022.107,985.137,7.470,1.627,0.215,4.19,1.87,981108
4243283,9811.0,9,40,4000,1,996.468,986.201,7.374,-0.178,-0.024,4.64,1.90,981109
4243284,9811.0,10,40,4000,1,1019.894,1009.985,-7.780,-0.610,-3.063,4.72,1.87,981110
4243285,9811.0,11,40,4000,1,1028.592,1022.465,-1.619,-1.729,-2.324,4.49,1.83,981111


### Total number of cases

In [3]:
unique_cases = df_train['case_id'].nunique()
unique_cases


9811

### Use small amout of cases for testing

In [4]:
number_of_cases = 2300

#df_train_small = df_train[df_train['case_id'] <= number_of_cases]
df_train_small = df_train[(df_train['case_id'] > number_of_cases) & (df_train['case_id'] <= number_of_cases + 300)]
df_train_small 

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width,unique_id
1018038,2301.0,1,1,100,1,1037.796,990.762,0.000,0.000,2.281,4.51,1.91,230101
1018039,2301.0,2,1,100,1,1044.278,983.999,0.000,0.000,2.440,4.31,1.89,230102
1018040,2301.0,3,1,100,1,1038.715,1013.339,3.062,6.510,1.131,4.53,1.88,230103
1018041,2301.0,4,1,100,1,1034.172,1005.157,2.876,6.589,1.159,4.56,1.80,230104
1018042,2301.0,5,1,100,1,1050.710,979.166,0.000,0.000,2.546,4.92,1.91,230105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200049,2600.0,3,40,4000,1,1032.480,995.423,5.067,4.126,0.683,4.50,1.81,260003
1200050,2600.0,4,40,4000,1,1046.491,981.977,-0.407,0.262,2.569,4.78,1.93,260004
1200051,2600.0,5,40,4000,1,1017.211,986.657,7.388,1.435,0.192,4.97,1.94,260005
1200052,2600.0,6,40,4000,1,1042.193,968.252,-0.521,0.747,2.180,4.51,1.90,260006


In [5]:
v_max = max(df_train_small['vx']**2 + df_train_small['vy']**2) ** 0.5

In [6]:
# load uniqueTracks data
# uniqueTracks is a dictionary strore each unique object
uniqueTracks = dp.read_uniqueTracks(df_train_small)
uniqueTracks

{230101: <data_preprocessing.uniqueTrack at 0x13886aab0>,
 230102: <data_preprocessing.uniqueTrack at 0x138868e30>,
 230103: <data_preprocessing.uniqueTrack at 0x1388a5a30>,
 230104: <data_preprocessing.uniqueTrack at 0x138869070>,
 230105: <data_preprocessing.uniqueTrack at 0x1388a58e0>,
 230106: <data_preprocessing.uniqueTrack at 0x1388a5670>,
 230107: <data_preprocessing.uniqueTrack at 0x1388a4a40>,
 230108: <data_preprocessing.uniqueTrack at 0x1388a4da0>,
 230109: <data_preprocessing.uniqueTrack at 0x1388a5820>,
 230110: <data_preprocessing.uniqueTrack at 0x1388a5a90>,
 230111: <data_preprocessing.uniqueTrack at 0x1388a5be0>,
 230112: <data_preprocessing.uniqueTrack at 0x1388a5130>,
 230113: <data_preprocessing.uniqueTrack at 0x1388a4f20>,
 230114: <data_preprocessing.uniqueTrack at 0x1388a4500>,
 230115: <data_preprocessing.uniqueTrack at 0x1388a47a0>,
 230116: <data_preprocessing.uniqueTrack at 0x1388a4440>,
 230118: <data_preprocessing.uniqueTrack at 0x1388a4410>,
 230117: <data

In [7]:
# case 1: the ego is in the current time frame
ego_id = 1010
current_time = 100

# get other vehicles with same case and time
other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
print("other agents:", other_agents)
print("interactive agens:", interactive_agents)

other agents: []
interactive agens: []


In [8]:
total_agents = 0
non_car_agents = []
car_agents = []
collisions = []
for ego_id in uniqueTracks:
    total_agents += 1
    for current_time in range(100, 4001, 100):
        other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
        # print(ego_id)
        mdp_tuple = mdp.get_mdp_tuple(ego_id, interactive_agents, current_time, uniqueTracks, v_max)
        if not mdp_tuple: # if interactive_agents is empty or ego is not a car, then mdp_tuple is None
            #print(mdp_tuple)
            if ego_id not in non_car_agents:
                if uniqueTracks[ego_id].agent_type == -1:
                    non_car_agents.append(ego_id)
        else:
            if ego_id not in car_agents:
                if uniqueTracks[ego_id].agent_type == 1:
                    car_agents.append(ego_id)
            s, a, r, s_next, the_last_episode = mdp_tuple
            if r < 0:
                if ego_id not in collisions: 
                    collisions.append(ego_id)
                    #print(uniqueTracks[ego_id].agent_type)
                    #print(r)
                    #print(ego_id)
            if the_last_episode:
                break
print('Total agents: ', total_agents)
print('Total car agents: ', len(car_agents))
print('Total nor-car agents: ', len(non_car_agents))
print('Total collisions: ', collisions)

/Users/kycheng/Desktop/5_Winter2024/1_EECS 545/project/github/EECS-545-final-project/reward.py:123: RuntimeWarning: invalid value encountered in arccos
  if rotated_rectangles_intersect(((ego_x, ego_y), 0.95 * ego_length, 0.95 * ego_width, np.arccos(ego_psi)),
/Users/kycheng/Desktop/5_Winter2024/1_EECS 545/project/github/EECS-545-final-project/reward.py:124: RuntimeWarning: invalid value encountered in arccos
  ((interactive_x, interactive_y), 0.95 * interactive_length, 0.95 * interactive_width, np.arccos(interactive_psi))):


non_car_collision:  231201
non_car_collision:  231201
non_car_collision:  231201
non_car_collision:  231201
non_car_collision:  234301
non_car_collision:  234301
non_car_collision:  234301
non_car_collision:  244101
non_car_collision:  244101
non_car_collision:  244101
non_car_collision:  244101
non_car_collision:  251101
non_car_collision:  251101
non_car_collision:  251101
non_car_collision:  251101
Total agents:  5315
Total car agents:  5060
Total nor-car agents:  207
Total collisions:  [231204, 234304, 244104, 251104, 254208, 254209, 256006, 256007]


In [9]:
#print('Total number of car agents: ', len(df_train_small['unique_id'][df_train['agent_type'] == 1].unique()))
#print('Total number of non-car agents: ', len(df_train_small['unique_id'][df_train['agent_type'] == -1].unique()))

### edge case: ego appears at the last time frame

In [10]:
#df_train_small[df_train_small['unique_id'] == 1214]

In [11]:
'''
ego_id = 602
current_time = 100

# get other vehicles with same case and time
other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
print("other agents:", other_agents)
print("interactive agents:", interactive_agents)
'''

'\nego_id = 602\ncurrent_time = 100\n\n# get other vehicles with same case and time\nother_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)\nprint("other agents:", other_agents)\nprint("interactive agents:", interactive_agents)\n'

In [12]:
#df_train_small[df_train_small['unique_id'] == 602]

In [13]:
ego_id = 159803
current_time = 100

# get other vehicles with same case and time
other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
print("other agents:", other_agents)
print("interactive agens:", interactive_agents)

other agents: []
interactive agens: []


In [ ]:
# 231201
# 

In [16]:
data = df_train_small[df_train_small['unique_id'] == 231201]
data2 = df_train_small[df_train_small['unique_id'] == 231202]
data3 = df_train_small[df_train_small['unique_id'] == 231203]
data4 = df_train_small[df_train_small['unique_id'] == 231204]
#data5 = df_train_small[df_train_small['unique_id'] == 606]

fig, ax = plt.subplots()
xdata, ydata = [], []
xdata2, ydata2 = [], []
xdata3, ydata3 = [], []
xdata4, ydata4 = [], []
#xdata5, ydata5 = [], []
ln, = plt.plot([], [], 'ro')  # 'ro' 表示紅色的圓點
ln2, = plt.plot([], [], 'bo')
ln3, = plt.plot([], [], 'go')
ln4, = plt.plot([], [], 'ko')
#ln5, = plt.plot([], [], 'rs')

def init():
    ax.set_xlim(950, 1060)  # 根據數據的 x 值範圍設定
    ax.set_ylim(950, 1020)  # 根據數據的 y 值範圍設定
    return ln, ln2, ln3, ln4#, ln5

def update(frame):
    xdata.append(data.iloc[frame]['x'])
    ydata.append(data.iloc[frame]['y'])
    ln.set_data(xdata, ydata)

    if frame < len(data2):
        xdata2.append(data2.iloc[frame]['x'])
        ydata2.append(data2.iloc[frame]['y'])
        ln2.set_data(xdata2, ydata2)
    else:
        ln2.set_data(xdata2, ydata2)
    #'''
    if frame < len(data3):
        xdata3.append(data3.iloc[frame]['x'])
        ydata3.append(data3.iloc[frame]['y'])
        ln3.set_data(xdata3, ydata3)
    else:
        ln3.set_data(xdata3, ydata3)

    if frame < len(data4):
        xdata4.append(data4.iloc[frame]['x'])
        ydata4.append(data4.iloc[frame]['y'])
        ln4.set_data(xdata4, ydata4)
    else:
        ln4.set_data(xdata4, ydata4)
    '''
    if frame < len(data5):
        xdata5.append(data5.iloc[frame]['x'])
        ydata5.append(data5.iloc[frame]['y'])
        ln5.set_data(xdata5, ydata5)
    else:
        ln5.set_data(xdata5, ydata5)
    '''
    
    return ln, ln2, ln3, ln4#, ln5

ani = animation.FuncAnimation(fig, update, frames=range(len(data)),
                    init_func=init, blit=False)

#plt.show()
# To display the animation in a Jupyter notebook you can use the following:
plt.close(fig)  # Avoid displaying the static plot
from IPython.display import HTML
HTML(ani.to_jshtml())  # Convert the animation to JSHTML and display it
